In [ ]:
import os
import numpy as np
import tensorflow.keras as krs
from keras.datasets import mnist
from PIL import Image, ImageFilter

def imageprepare(argv):
    im = Image.open(argv, mode="r").convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28, 28), (255))  # creates white canvas of 28x28 pixels

    if width > height:
        nheight = int(round((20.0 / width * height), 0))
        if nheight == 0:
            nheight = 1
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight) / 2), 0))
        newImage.paste(img, (4, wtop))
    else:
        nwidth = int(round((20.0 / height * width), 0))
        if nwidth == 0:
            nwidth = 1
        img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))
        newImage.paste(img, (wleft, 4))

    tv = list(newImage.getdata())

    tva = [(255 - x) * 1.0 / 255.0 for x in tv]
    return tva

# Load MNIST dataset
(trainx, trainy), (testx, testy) = mnist.load_data()
all_image = (trainx/255.0-0.5)*1.999
all_image = np.expand_dims(all_image, axis=3)

# Define encoder model
nw = nh = 28
num_hide = 98
encoder_input = krs.layers.Input(shape=(nw, nh, 1))
lay = krs.layers.Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(encoder_input)
lay = krs.layers.Dropout(0.15)(lay)
lay = krs.layers.Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same')(lay)
lay = krs.layers.Dropout(0.15)(lay)
lay = krs.layers.Conv2D(128, (3, 3), strides=(2, 2), activation='relu', padding='same')(lay)
lay = krs.layers.Dropout(0.15)(lay)
lay = krs.layers.Conv2D(256, (3, 3), strides=(2, 2), activation='relu', padding='same')(lay)
lay = krs.layers.Flatten()(lay)
lay_out_encoder = krs.layers.Dense(num_hide, activation="linear", name='den4')(lay)
encoder = krs.Model(encoder_input, lay_out_encoder)
encoder = krs.models.load_model("encoder_model.h5")


out_vec = encoder.predict(all_image)

# Define classifier model
num_classes = 10
classifier_input = krs.layers.Input(shape=(num_hide,))
lay = krs.layers.Dense(128, activation='relu')(classifier_input)
lay = krs.layers.Dropout(0.2)(lay)
lay = krs.layers.Dense(256, activation='relu')(lay)
lay = krs.layers.Dropout(0.2)(lay)
lay = krs.layers.Dense(512, activation='relu')(lay)
lay = krs.layers.Dropout(0.2)(lay)
lay = krs.layers.Dense(256, activation='relu')(lay)
lay = krs.layers.Dropout(0.2)(lay)
lay = krs.layers.Dense(128, activation='relu')(lay)
lay = krs.layers.BatchNormalization()(lay)
lay = krs.layers.Dense(10, activation='relu', kernel_initializer='random_uniform')(lay)
classifier_output = krs.layers.Dense(num_classes, activation="softmax", name='den4')(lay)
classificator = krs.Model(classifier_input, classifier_output)

# Compile the classificator model
classificator.compile(loss='categorical_crossentropy',
                      optimizer=krs.optimizers.Adam(learning_rate=0.0002),
                      metrics=['accuracy', krs.metrics.Precision(), krs.metrics.Recall()])

# Now you can call the fit method to train the model
classificator.fit(x=out_vec, y=all_out, batch_size=8000, epochs=ep * 2)

# Load trained weights

# Directory containing the images
image_dir = '/content/sample_data/images/raw/'

for i in range(10):
    for filename in os.listdir(f'{image_dir}/{i}'):
        img_path = os.path.join(f'{image_dir}/{i}', filename)
        x = [imageprepare(img_path)]  # This returns a list of length 1
        eps = np.array(x)  # Convert the list to a numpy array
        eps = eps.reshape(-1, 28, 28, 1)  # Reshape to match the input shape expected by the encoder
        out_vec = encoder.predict(eps)  # Predict using the encoder model
        predictions = classificator.predict(out_vec)
        predicted_label = np.argmax(predictions)
        print(f"Image: {img_path}, Predicted Label: {predicted_label}")


1875/1875 [==============================] - 3s 2ms/step
Epoch 1/80
8/8 [==============================] - 3s 20ms/step - loss: 2.2705 - accuracy: 0.1512 - precision_24: 0.0000e+00 - recall_24: 0.0000e+00
Epoch 2/80
8/8 [==============================] - 0s 19ms/step - loss: 2.1311 - accuracy: 0.3151 - precision_24: 0.0000e+00 - recall_24: 0.0000e+00
Epoch 3/80
8/8 [==============================] - 0s 17ms/step - loss: 1.9865 - accuracy: 0.4274 - precision_24: 1.0000 - recall_24: 2.3333e-04
Epoch 4/80
8/8 [==============================] - 0s 14ms/step - loss: 1.8486 - accuracy: 0.4831 - precision_24: 0.9704 - recall_24: 0.0093
Epoch 5/80
8/8 [==============================] - 0s 13ms/step - loss: 1.7217 - accuracy: 0.5198 - precision_24: 0.9754 - recall_24: 0.0422
Epoch 6/80
8/8 [==============================] - 0s 13ms/step - loss: 1.6088 - accuracy: 0.5546 - precision_24: 0.9684 - recall_24: 0.0910
Epoch 7/80
8/8 [==============================] - 0s 13ms/step - loss: 1.5026 - acc

<ipython-input-19-7b5e412a6c4c>:24: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)


1/1 [==============================] - 0s 126ms/step
Image: /content/sample_data/images/raw//0/test_image0.jpg, Predicted Label: 1
1/1 [==============================] - 0s 27ms/step
Image: /content/sample_data/images/raw//0/test_image1.jpg, Predicted Label: 8
1/1 [==============================] - 0s 24ms/step
Image: /content/sample_data/images/raw//0/test_image4.jpg, Predicted Label: 8
1/1 [==============================] - 0s 28ms/step
Image: /content/sample_data/images/raw//0/test_image3.jpg, Predicted Label: 5
1/1 [==============================] - 0s 25ms/step
Image: /content/sample_data/images/raw//0/test_image6.jpg, Predicted Label: 8
1/1 [==============================] - 0s 24ms/step
Image: /content/sample_data/images/raw//0/test_image5.jpg, Predicted Label: 8
1/1 [==============================] - 0s 24ms/step
Image: /content/sample_data/images/raw//0/test_image2.jpg, Predicted Label: 8
1/1 [==============================] - 0s 24ms/step
Image: /content/sample_data/images/ra

<ipython-input-19-7b5e412a6c4c>:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)


1/1 [==============================] - 0s 43ms/step
Image: /content/sample_data/images/raw//1/test_image19.jpg, Predicted Label: 8
1/1 [==============================] - 0s 30ms/step
Image: /content/sample_data/images/raw//1/test_image3.jpg, Predicted Label: 1
1/1 [==============================] - 0s 30ms/step
Image: /content/sample_data/images/raw//1/test_image12.jpg, Predicted Label: 1
1/1 [==============================] - 0s 27ms/step
Image: /content/sample_data/images/raw//1/test_image24.jpg, Predicted Label: 1
1/1 [==============================] - 0s 26ms/step
Image: /content/sample_data/images/raw//1/test_image6.jpg, Predicted Label: 1
1/1 [==============================] - 0s 16ms/step
Image: /content/sample_data/images/raw//1/test_image5.jpg, Predicted Label: 8
1/1 [==============================] - 0s 21ms/step
Image: /content/sample_data/images/raw//1/test_image15.jpg, Predicted Label: 1
1/1 [==============================] - 0s 17ms/step
Image: /content/sample_data/images

In [ ]:
# MIT License
# Copyright (c) 2023 saysaysx

import matplotlib.pyplot as plt
import tensorflow.keras as krs
import numpy
from keras.datasets import mnist
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
nw = 28
nh = 28
num_hide = 98
# загружаем примеры обучения mnist (рукописные цифры)
(trainx, trainy), (testx, testy) = mnist.load_data()
# нормируем от -1 до 1 изображения цифр
all_image = (trainx/255.0-0.5)*1.999
# добавляем дополнительное измерение соответствующее одной цветовой карте
all_image = numpy.expand_dims(all_image, axis=3)
# задаем входной слой экодера высота на ширину на количество карт
encoder_input = krs.layers.Input(shape=(nw,nh,1))
# задаем сверточный слой с 32 фильтрами-картами и фильтрами 3 на 3
# оставляет тот же размер карты 28*28
lay = krs.layers.Conv2D(32, (3, 3), strides = (2,2), activation='relu', padding='same')(encoder_input)
lay = krs.layers.Dropout(0.15)(lay)
lay = krs.layers.Conv2D(64, (3, 3), strides = (2,2),activation='relu', padding='same')(lay)
# добавляем слой прореживания
lay = krs.layers.Dropout(0.15)(lay)
lay = krs.layers.Conv2D(128, (3, 3), strides = (2,2),activation='relu', padding='same')(lay)
lay = krs.layers.Dropout(0.15)(lay)
lay = krs.layers.Conv2D(256, (3, 3), strides = (2,2),activation='relu', padding='same')(lay)
# слой который многомерный тензорный слой превращает в плоский вектор
lay = krs.layers.Flatten()(lay)
# выходной кодирующий слой
lay_out_encoder = krs.layers.Dense(num_hide, activation="linear", name='den4')(lay)
# создаем сеть энкодера
encoder = krs.Model(encoder_input, lay_out_encoder)

# создание сети декодера, входной слой
decoder_input = krs.layers.Input(shape=(num_hide,))
lay = krs.layers.Dense(128*7*7)(decoder_input)
# преобразуем плоский слой в многомерный тензор 7*7*128
lay = krs.layers.Reshape(target_shape=(7,7,128))(lay)
lay = krs.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(lay)
# повышаем размерность карты в два раза, будет 14*14
# можно использовать билинейную интерполяцию если хотите
lay = krs.layers.UpSampling2D(size=(2,2))(lay)
lay = krs.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(lay)
lay = krs.layers.UpSampling2D(size=(2,2))(lay)
lay_out_decoder = krs.layers.Conv2D(1, (3, 3), activation='tanh', padding='same')(lay)
# создаем сеть декодера
decoder = krs.Model(decoder_input,lay_out_decoder)

# объединяем обе сети в автоэнкодер
lay_out = decoder(lay_out_encoder)
autoencoder = krs.Model(encoder_input,lay_out)
# полученный вид модели сохраняем в файле в виде изображения
krs.utils.plot_model(autoencoder, to_file='.\out\autoencoder.png', show_shapes=True)
# компилируем модель автоэнкодера с функцией потерь mse и скоростью обучения 0.0002
autoencoder.compile(loss='mean_squared_error', optimizer=krs.optimizers.Adam(learning_rate=0.0002),
                  metrics=['accuracy', krs.metrics.Precision(), krs.metrics.Recall()])
# запускаем 40 эпох обучения с размером батча 4000
ep = 40
autoencoder.fit(x = all_image,y = all_image,batch_size = 4000,epochs = ep)
encoder.save('encoder_model.h5')

Epoch 1/40
15/15 [==============================] - 43s 331ms/step - loss: 0.7084 - accuracy: 0.0000e+00 - precision_20: 0.0000e+00 - recall_20: 0.0000e+00
Epoch 2/40
15/15 [==============================] - 5s 336ms/step - loss: 0.3368 - accuracy: 0.0000e+00 - precision_20: 0.0000e+00 - recall_20: 0.0000e+00
Epoch 3/40
15/15 [==============================] - 5s 342ms/step - loss: 0.2776 - accuracy: 0.0000e+00 - precision_20: 0.0000e+00 - recall_20: 0.0000e+00
Epoch 4/40
15/15 [==============================] - 5s 342ms/step - loss: 0.2694 - accuracy: 0.0000e+00 - precision_20: 0.0000e+00 - recall_20: 0.0000e+00
Epoch 5/40
15/15 [==============================] - 5s 329ms/step - loss: 0.2670 - accuracy: 0.0000e+00 - precision_20: 0.0000e+00 - recall_20: 0.0000e+00
Epoch 6/40
15/15 [==============================] - 5s 325ms/step - loss: 0.2653 - accuracy: 0.0000e+00 - precision_20: 0.0000e+00 - recall_20: 0.0000e+00
Epoch 7/40
15/15 [==============================] - 5s 322ms/step - l

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')